<a href="https://colab.research.google.com/github/harish143coding/DaSciUd/blob/develop/Pyspark_learnings_nb1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Executor cell


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, when, avg, max, min, count, row_number, rank, dense_rank, lag
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("learningsession").getOrCreate()

In [2]:
#start with D12
data = [("a", "el", 11),
        ("b", "el", 12),
        ("c", "fur", 13),
        ("d", "fur", 14),
        ("e", "el", 15),
        ("f", "fur", 16),
        ("e","fur", 22),
        ("f", "el", 24)
]
df  = spark.createDataFrame(data, ["Customer", "category", "Amount"])
df.show()

+--------+--------+------+
|Customer|category|Amount|
+--------+--------+------+
|       a|      el|    11|
|       b|      el|    12|
|       c|     fur|    13|
|       d|     fur|    14|
|       e|      el|    15|
|       f|     fur|    16|
|       e|     fur|    22|
|       f|      el|    24|
+--------+--------+------+



In [ ]:
df.groupBy("category").sum("amount").show()

+--------+-----------+
|category|sum(amount)|
+--------+-----------+
|      el|         38|
|     fur|         43|
+--------+-----------+



In [ ]:
df.groupBy("customer").sum("amount").show()

+--------+-----------+
|customer|sum(amount)|
+--------+-----------+
|       c|         13|
|       b|         12|
|       a|         11|
|       f|         16|
|       e|         15|
|       d|         14|
+--------+-----------+



In [ ]:
#cntd D12S3
df.groupBy("Customer").agg(sum("Amount").alias("Totalspent"),\
        avg("Amount").alias("Avgspent"),\
        max("Amount").alias("Maxspent"),\
        min("Amount").alias("Minspent")).show()


+--------+----------+--------+--------+--------+
|Customer|Totalspent|Avgspent|Maxspent|Minspent|
+--------+----------+--------+--------+--------+
|       c|        13|    13.0|      13|      13|
|       b|        12|    12.0|      12|      12|
|       a|        11|    11.0|      11|      11|
|       f|        16|    16.0|      16|      16|
|       e|        15|    15.0|      15|      15|
|       d|        14|    14.0|      14|      14|
+--------+----------+--------+--------+--------+



In [ ]:
df.groupBy("Customer", "category").sum("Amount").show()

+--------+--------+-----------+
|Customer|category|sum(Amount)|
+--------+--------+-----------+
|       a|      el|         11|
|       c|     fur|         13|
|       b|      el|         12|
|       f|     fur|         16|
|       d|     fur|         14|
|       e|      el|         15|
+--------+--------+-----------+



In [ ]:
cat_agg = df.groupBy("category").agg(\
             sum("Amount").alias("Totalspent"),\
             avg("Amount").alias("Avgspent"),\
             count("*").alias("Totalcount")\
             )
cat_agg.show()

+--------+----------+------------------+----------+
|category|Totalspent|          Avgspent|Totalcount|
+--------+----------+------------------+----------+
|      el|        38|12.666666666666666|         3|
|     fur|        43|14.333333333333334|         3|
+--------+----------+------------------+----------+



In [ ]:
cat_cus_df = df.groupBy("Customer", "category")\
               .agg(sum("Amount").alias("Totalspent"),\
                    max("Amount").alias("Maxamount"))
cat_cus_df.show()

+--------+--------+----------+---------+
|Customer|category|Totalspent|Maxamount|
+--------+--------+----------+---------+
|       a|      el|        11|       11|
|       c|     fur|        13|       13|
|       b|      el|        12|       12|
|       f|     fur|        16|       16|
|       d|     fur|        14|       14|
|       e|      el|        15|       15|
+--------+--------+----------+---------+



In [4]:
# D13 start with widow functs, window_spec
window_spec = Window.partitionBy("Customer").orderBy(df['Amount'])
type(window_spec)

pyspark.sql.window.WindowSpec

In [5]:
# ranking the DF
df_ranks = df.withColumn("row_number", row_number().over(window_spec))\
             .withColumn("rank", rank().over(window_spec))\
             .withColumn("dense_rank", dense_rank().over(window_spec))
df_ranks.show()

+--------+--------+------+----------+----+----------+
|Customer|category|Amount|row_number|rank|dense_rank|
+--------+--------+------+----------+----+----------+
|       a|      el|    11|         1|   1|         1|
|       b|      el|    12|         1|   1|         1|
|       c|     fur|    13|         1|   1|         1|
|       d|     fur|    14|         1|   1|         1|
|       e|      el|    15|         1|   1|         1|
|       e|     fur|    22|         2|   2|         2|
|       f|     fur|    16|         1|   1|         1|
|       f|      el|    24|         2|   2|         2|
+--------+--------+------+----------+----+----------+



In [8]:
df_lag = df.withColumn("lag_amount", lag("Amount").over(window_spec))
df_lag.show()

+--------+--------+------+----------+
|Customer|category|Amount|lag_amount|
+--------+--------+------+----------+
|       a|      el|    11|      NULL|
|       b|      el|    12|      NULL|
|       c|     fur|    13|      NULL|
|       d|     fur|    14|      NULL|
|       e|      el|    15|      NULL|
|       e|     fur|    22|        15|
|       f|     fur|    16|      NULL|
|       f|      el|    24|        16|
+--------+--------+------+----------+

